# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging

import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


C:\Users\Zara\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [Movie](http://www.cs.cmu.edu/~ark/personas/) dataset. 42,306 movie plot summaries extracted from Wikipedia + aligned metadata extracted from Freebase, including:
Movie box office revenue, genre, release date, runtime, and language
Character names and aligned information about the actors who portray them, including gender and estimated age at the time of the movie's release.
We will use the compressed file for this tutorial. Each line in this file represents a movie plot. 
To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="plot_summaries.gz"

with gzip.open ('plot_summaries.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"plot_summaries.txt\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000100777\x000000000\x000000000\x0000441524521\x0013356553414\x00011653\x00 0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ustar\x0000\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [3]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2018-10-11 12:30:01,713 : INFO : reading file plot_summaries.gz...this may take a while
2018-10-11 12:30:01,718 : INFO : read 0 reviews
2018-10-11 12:30:15,926 : INFO : read 10000 reviews
2018-10-11 12:30:29,224 : INFO : read 20000 reviews
2018-10-11 12:30:41,962 : INFO : read 30000 reviews
2018-10-11 12:30:54,471 : INFO : read 40000 reviews
2018-10-11 12:30:57,356 : INFO : Done reading data file


## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2018-10-11 12:31:27,654 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-10-11 12:31:27,658 : INFO : collecting all words and their counts
2018-10-11 12:31:27,661 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-11 12:31:29,344 : INFO : PROGRESS: at sentence #10000, processed 3053049 words, keeping 66592 word types
2018-10-11 12:31:31,008 : INFO : PROGRESS: at sentence #20000, processed 6091040 words, keeping 94173 word types
2018-10-11 12:31:32,747 : INFO : PROGRESS: at sentence #30000, processed 9090681 words, keeping 115640 word types
2018-10-11 12:31:34,282 : INFO : PROGRESS: at sentence #40000, processed 12053234 words, keeping 134163 word types
2018-10-11 12:31:34,658 : INFO : collected 138147 word types from a corpus of 12747456 raw words and 42307 sentences
2018-10-11 12:31:34,661 : INFO : Loading a fresh vocabulary
2018-10-11 12:31:36,101 : INFO : min_count=2 retains 89821 unique words (65% of original 13

2018-10-11 12:32:21,892 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-10-11 12:32:21,919 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-10-11 12:32:21,923 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-10-11 12:32:21,931 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-10-11 12:32:21,943 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-10-11 12:32:21,951 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-10-11 12:32:21,964 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-10-11 12:32:21,976 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-10-11 12:32:21,987 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-10-11 12:32:21,991 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-10-11 12:32:21,992 : INFO : EPOCH - 3 : training on 12747456 raw words (9683124 effec

2018-10-11 12:32:59,061 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-10-11 12:32:59,067 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-10-11 12:32:59,069 : INFO : EPOCH - 1 : training on 12747456 raw words (9683041 effective words) took 11.8s, 821254 effective words/s
2018-10-11 12:33:00,088 : INFO : EPOCH 2 - PROGRESS: at 8.14% examples, 798489 words/s, in_qsize 19, out_qsize 0
2018-10-11 12:33:01,100 : INFO : EPOCH 2 - PROGRESS: at 16.89% examples, 827500 words/s, in_qsize 20, out_qsize 0
2018-10-11 12:33:02,103 : INFO : EPOCH 2 - PROGRESS: at 25.78% examples, 834352 words/s, in_qsize 19, out_qsize 0
2018-10-11 12:33:03,116 : INFO : EPOCH 2 - PROGRESS: at 34.54% examples, 836068 words/s, in_qsize 19, out_qsize 0
2018-10-11 12:33:04,135 : INFO : EPOCH 2 - PROGRESS: at 43.30% examples, 837788 words/s, in_qsize 17, out_qsize 2
2018-10-11 12:33:05,161 : INFO : EPOCH 2 - PROGRESS: at 52.20% examples, 839426 words/s, in_qsize 19, out_qsiz

2018-10-11 12:33:44,493 : INFO : EPOCH 5 - PROGRESS: at 75.01% examples, 723684 words/s, in_qsize 18, out_qsize 1
2018-10-11 12:33:45,506 : INFO : EPOCH 5 - PROGRESS: at 83.92% examples, 733208 words/s, in_qsize 17, out_qsize 2
2018-10-11 12:33:46,507 : INFO : EPOCH 5 - PROGRESS: at 92.71% examples, 741368 words/s, in_qsize 18, out_qsize 1
2018-10-11 12:33:47,312 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-10-11 12:33:47,336 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-10-11 12:33:47,340 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-10-11 12:33:47,348 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-10-11 12:33:47,365 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-10-11 12:33:47,370 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-10-11 12:33:47,376 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-10-11 12:33:47,3

2018-10-11 12:34:23,644 : INFO : EPOCH - 8 : training on 12747456 raw words (9682689 effective words) took 12.4s, 779385 effective words/s
2018-10-11 12:34:24,675 : INFO : EPOCH 9 - PROGRESS: at 7.01% examples, 681571 words/s, in_qsize 18, out_qsize 1
2018-10-11 12:34:25,680 : INFO : EPOCH 9 - PROGRESS: at 15.14% examples, 742185 words/s, in_qsize 19, out_qsize 0
2018-10-11 12:34:26,681 : INFO : EPOCH 9 - PROGRESS: at 23.82% examples, 769933 words/s, in_qsize 18, out_qsize 1
2018-10-11 12:34:27,690 : INFO : EPOCH 9 - PROGRESS: at 30.75% examples, 747097 words/s, in_qsize 18, out_qsize 1
2018-10-11 12:34:28,744 : INFO : EPOCH 9 - PROGRESS: at 37.42% examples, 720621 words/s, in_qsize 17, out_qsize 2
2018-10-11 12:34:29,753 : INFO : EPOCH 9 - PROGRESS: at 43.30% examples, 694269 words/s, in_qsize 20, out_qsize 0
2018-10-11 12:34:30,757 : INFO : EPOCH 9 - PROGRESS: at 49.32% examples, 677984 words/s, in_qsize 18, out_qsize 1
2018-10-11 12:34:31,758 : INFO : EPOCH 9 - PROGRESS: at 56.70% e

(96829269, 127474560)

In [5]:
#model.accuracy("plot_summaries.txt")

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [6]:

w1 = "joe"
model.wv.most_similar (positive=w1)


2018-10-11 12:35:58,153 : INFO : precomputing L2-norms of word weight vectors


[('billy', 0.5901859998703003),
 ('mike', 0.5875735282897949),
 ('jimmy', 0.5773594975471497),
 ('pat', 0.5625245571136475),
 ('bill', 0.5478695034980774),
 ('buck', 0.5374084115028381),
 ('gene', 0.5347920656204224),
 ('jack', 0.5299627780914307),
 ('jill', 0.5206817984580994),
 ('eddie', 0.5166523456573486)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [39]:
# look up top 6 words similar to 'polite'
w1 = ["happy"]
model.wv.most_similar (positive=w1,topn=6)


[('content', 0.5697418451309204),
 ('pleased', 0.5660367012023926),
 ('nice', 0.5551778078079224),
 ('comfortable', 0.5461371541023254),
 ('glad', 0.5310183763504028),
 ('thrilled', 0.5267074108123779)]

In [43]:
# look up top 6 words similar to 'france'
w1 = ["science"]
model.wv.most_similar (positive=w1,topn=6)


[('mathematics', 0.6450624465942383),
 ('biology', 0.6266740560531616),
 ('physics', 0.6115038394927979),
 ('genetics', 0.5985028743743896),
 ('psychology', 0.5966692566871643),
 ('study', 0.5859565734863281)]

In [10]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('surprised', 0.7801766395568848),
 ('horrified', 0.7585006356239319),
 ('stunned', 0.7455542683601379),
 ('overjoyed', 0.6618738770484924),
 ('astonished', 0.6385260224342346),
 ('amazed', 0.6269989609718323)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [11]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('sheets', 0.645630955696106),
 ('cloth', 0.6073095798492432),
 ('drawer', 0.5755912661552429),
 ('wrists', 0.5676925182342529),
 ('needle', 0.5640305876731873),
 ('lipstick', 0.5587670207023621),
 ('bathtub', 0.5573904514312744),
 ('coat', 0.5570542812347412),
 ('fireplace', 0.5559794902801514),
 ('shade', 0.5515347123146057)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [7]:
# similarity between two different words
model.wv.similarity(w1="joe",w2="bill")

0.54786948833762228

In [13]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [14]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.24250703451570818

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [15]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [16]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'duvet'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
